<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# SQL Server 2019 big data cluster Tutorial
## 02 - Data Virtualization

In this tutorial you will learn how to create and query Virtualized Data in a SQL Server big data cluster. 

**TODO:** Complete Tutorial 
